In [18]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split, DataLoader, Dataset, TensorDataset

In [19]:
"""
dataset
"""

# data loading
dataFile = pd.read_excel(r'iris_dataset.xlsx')
print('Take a look at sample from the dataset:')
print(dataFile.head())

# data balance verifing
print('\n')
print('Our dataset is balanced and has the following values to predict:')
print(dataFile['Iris_Type'].value_counts())

# data washing
labels = {'Iris-setosa':0, 'Iris-virginica':1, 'Iris-versicolor':2}
dataFile['IrisType_num'] = dataFile['Iris_Type']  # create a new column
dataFile.IrisType_num = [labels[item] for item in dataFile.IrisType_num]

# define input and output
print('\n')
input = dataFile.iloc[:, 1:-2]  # drop the 1st column and the last two
print('Input values are:')
print(input.head())
print('\n')
output = dataFile.loc[:, 'IrisType_num']  # output the last column
print('Output value is:')
print(output.head())

# convert input and output to tensors
print('\n')
input = torch.tensor(input.to_numpy()).float()  # create tensor type torch.float32
print('Input format: ', input.shape, input.dtype)
output = torch.tensor(output.to_numpy())        # create tensor type torch.int64
print('Output format: ', output.shape, output.dtype)
data = TensorDataset(input, output)

# split into train, verify and test
number_rows = len(input)
test_split = int(number_rows * 0.3)
verify_split = int(number_rows * 0.2)
train_split = number_rows - test_split - verify_split
train_set, verify_set, test_set = random_split(data, [train_split, verify_split, test_split])

# create DataLoader
train_batch_size = 10
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True)
verify_loader = DataLoader(verify_set, batch_size=1)
test_loader = DataLoader(test_set, batch_size=1)

Take a look at sample from the dataset:
     #  sepal length in cm  sepal width in cm  petal length in cm   
0    1                 5.1                3.5                 1.4  \
1  101                 6.3                3.3                 6.0   
2  110                 7.2                3.6                 6.1   
3  145                 6.7                3.3                 5.7   
4  115                 5.8                2.8                 5.1   

   petal width in cm       Iris_Type  
0                0.2     Iris-setosa  
1                2.5  Iris-virginica  
2                2.5  Iris-virginica  
3                2.5  Iris-virginica  
4                2.4  Iris-virginica  


Our dataset is balanced and has the following values to predict:
Iris_Type
Iris-setosa        50
Iris-virginica     50
Iris-versicolor    50
Name: count, dtype: int64


Input values are:
   sepal length in cm  sepal width in cm  petal length in cm   
0                 5.1                3.5                 1

In [20]:
"""
model
"""

class IrisModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(IrisModel, self).__init__()
        self.layer1 = nn.Linear(input_size, 24)
        self.layer2 = nn.Linear(24, 24)
        self.layer3 = nn.Linear(24, output_size)
    
    def forward(self, x):
        x1 = F.relu(self.layer1(x))
        x2 = F.relu(self.layer2(x1))
        x3 = self.layer3(x2)
        return x3
    
    def save(path):
        torch.save(self.state_dict(), path)
        
    # export for java calling
    def export(path):
        traced_script = torch.jit.trace(self, torch.zeros(input_size).float())

In [21]:
"""
init
"""

# select device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("The model will be running on", device, "device")

# create model
input_size = list(input.shape)[1]
output_size = len(labels)
model = IrisModel(input_size, output_size)
model.to(device)

# define loss-function
loss_fn = nn.CrossEntropyLoss(reduction='sum')

# define optimizer
LEARNING_RATE = 0.01
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

The model will be running on cpu device


In [29]:
def saveModel():
    torch.save(model.state_dict(), "iris-model.pth")
    # export model or java calling
    traced_script = torch.jit.trace(model, torch.tensor([1.0, 2.0, 3.0, 4.0]).float())
    traced_script.save("iris-model.out")

In [30]:
"""
train
"""

def train(num_epochs):
    accuracy_best = 0.0
    
    print("Begin training...")
    for epoch in range(1, num_epochs+1):
        running_train_loss = 0.0
        running_accuracy = 0.0
        running_val_loss = 0.0
        total = 0
        
        # train loop
        for data in train_loader:
            inputs, outputs = data
            pred_outputs = model(inputs)
            train_loss = loss_fn(pred_outputs, outputs)
            
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            running_train_loss += train_loss.item()
            
        # calculate train loss
        train_loss_value = running_train_loss / len(train_loader)
        
        # verify loop
        with torch.no_grad():
            model.eval()
            for data in verify_loader:
                inputs, outputs = data
                print('[verify] output:', outputs)
                pred_outputs = model(inputs)
                print('[verify] pred_output:', pred_outputs)
                val_loss = loss_fn(pred_outputs, outputs)
                print('[verify] val_loss:', val_loss)
                
                # the lavel with the highest value will be our prediction
                dummy, pred = torch.max(pred_outputs, 1)
                print('[verify] torch.max:', dummy, pred)
                
                running_val_loss += val_loss.item()
                total += outputs.size(0)
                running_accuracy += (pred == outputs).sum().item()
        
        # calculate validation loss value
        val_loss_value = running_val_loss / len(verify_loader)
        
        # calculate accuracy as the number of correct predictions in the validation batch divided by the total number of predictions done
        accuracy = 100 * (running_accuracy / total)
        
        # save the model if the accuracy is the best
        if accuracy > accuracy_best:
            saveModel()
            accuracy_best = accuracy
        
        # print the statistics of the epoch
        print('Completed training batch', epoch, 'Training Loss is: %.4f' %train_loss_value, 'Validation Loss is: %.4f' %val_loss_value, 'Accuracy is: %d %%' %accuracy)

In [34]:
"""

"""
# Function to test the model
def test():
    # Load the model that we saved at the end of the training loop
    model = IrisModel(input_size, output_size)
    path = "iris-model.pth"
    model.load_state_dict(torch.load(path))
    
    running_accuracy = 0
    total = 0
    
    with torch.no_grad():
        for data in test_loader:
            inputs, outputs = data
            outputs = outputs.to(torch.float32)
            pred_outputs = model(inputs)
            _, pred = torch.max(pred_outputs, 1)
            total += outputs.size(0)
            running_accuracy += (pred == outputs).sum().item()
        accuracy = 100 * (running_accuracy / total)
        print('Accuracy of model based on the best set of', test_split, 'inputs is: %d %%' %accuracy)

In [45]:
# Optional: Function to test which species were easier to predict
def test_species():
    # Load the model that we saved at the end of the training loop
    model = IrisModel(input_size, output_size)
    path = "iris-model.pth"
    model.load_state_dict(torch.load(path))
    
    labels_length = len(labels)  # how many labels of Irises we have
    labels_correct = list(0. for i in range(labels_length))
    labels_total = list(0. for i in range(labels_length))
    
    with torch.no_grad():
        for data in test_loader:
            inputs, outputs = data
            pred_outputs = model(inputs)
            _, pred = torch.max(pred_outputs, 1)
            
            label_correct_running = (pred == outputs).squeeze();
            label = outputs[0]
            if label_correct_running.item():
                labels_correct[label] += 1
            labels_total[label] += 1
    
    label_list = list(labels.keys())
    for i in range(output_size):
        print('Accuracy to predict %5s: %2d %%' % (label_list[i], 100 * labels_correct[i] / labels_total[i]))

In [46]:
if __name__ == "__main__":
    num_epochs = 10
    train(num_epochs)
    print('Finished Training\n')
    test()
    test_species()

Begin training...
[verify] output: tensor([0])
[verify] pred_output: tensor([[ 11.2245, -20.3975,   1.9669]])
[verify] val_loss: tensor(9.5363e-05)
[verify] torch.max: tensor([11.2245]) tensor([0])
[verify] output: tensor([0])
[verify] pred_output: tensor([[ 12.5081, -22.5802,   2.0858]])
[verify] val_loss: tensor(2.9802e-05)
[verify] torch.max: tensor([12.5081]) tensor([0])
[verify] output: tensor([2])
[verify] pred_output: tensor([[-3.9310, -5.0323,  3.6782]])
[verify] val_loss: tensor(0.0007)
[verify] torch.max: tensor([3.6782]) tensor([2])
[verify] output: tensor([0])
[verify] pred_output: tensor([[ 12.3692, -22.3780,   2.0978]])
[verify] val_loss: tensor(3.4570e-05)
[verify] torch.max: tensor([12.3692]) tensor([0])
[verify] output: tensor([0])
[verify] pred_output: tensor([[ 11.2437, -20.4941,   2.0122]])
[verify] val_loss: tensor(9.7866e-05)
[verify] torch.max: tensor([11.2437]) tensor([0])
[verify] output: tensor([0])
[verify] pred_output: tensor([[ 12.2705, -22.2454,   2.1120]]

[verify] val_loss: tensor(3.0875e-05)
[verify] torch.max: tensor([12.4016]) tensor([0])
[verify] output: tensor([2])
[verify] pred_output: tensor([[-4.6414, -4.1347,  3.3156]])
[verify] val_loss: tensor(0.0009)
[verify] torch.max: tensor([3.3156]) tensor([2])
[verify] output: tensor([0])
[verify] pred_output: tensor([[ 12.2835, -22.2176,   2.0286]])
[verify] val_loss: tensor(3.5166e-05)
[verify] torch.max: tensor([12.2835]) tensor([0])
[verify] output: tensor([0])
[verify] pred_output: tensor([[ 11.1617, -20.3446,   1.9507]])
[verify] val_loss: tensor(9.9892e-05)
[verify] torch.max: tensor([11.1617]) tensor([0])
[verify] output: tensor([0])
[verify] pred_output: tensor([[ 12.1826, -22.0832,   2.0441]])
[verify] val_loss: tensor(3.9577e-05)
[verify] torch.max: tensor([12.1826]) tensor([0])
[verify] output: tensor([0])
[verify] pred_output: tensor([[ 11.3786, -20.6272,   1.9117]])
[verify] val_loss: tensor(7.7364e-05)
[verify] torch.max: tensor([11.3786]) tensor([0])
[verify] output: ten

[verify] torch.max: tensor([2.8403]) tensor([2])
[verify] output: tensor([0])
[verify] pred_output: tensor([[ 11.8158, -21.2104,   1.8265]])
[verify] val_loss: tensor(4.5895e-05)
[verify] torch.max: tensor([11.8158]) tensor([0])
[verify] output: tensor([1])
[verify] pred_output: tensor([[-10.8377,   4.6507,  -0.6050]])
[verify] val_loss: tensor(0.0052)
[verify] torch.max: tensor([4.6507]) tensor([1])
[verify] output: tensor([2])
[verify] pred_output: tensor([[-4.4210, -5.7038,  4.1797]])
[verify] val_loss: tensor(0.0002)
[verify] torch.max: tensor([4.1797]) tensor([2])
[verify] output: tensor([1])
[verify] pred_output: tensor([[-11.8567,   5.3773,  -0.8745]])
[verify] val_loss: tensor(0.0019)
[verify] torch.max: tensor([5.3773]) tensor([1])
[verify] output: tensor([1])
[verify] pred_output: tensor([[-13.3183,   6.1934,  -1.1159]])
[verify] val_loss: tensor(0.0007)
[verify] torch.max: tensor([6.1934]) tensor([1])
[verify] output: tensor([1])
[verify] pred_output: tensor([[-12.3182,   5.

[verify] pred_output: tensor([[ 11.5119, -20.6594,   1.7145]])
[verify] val_loss: tensor(5.5550e-05)
[verify] torch.max: tensor([11.5119]) tensor([0])
[verify] output: tensor([0])
[verify] pred_output: tensor([[ 12.5651, -22.4235,   1.7841]])
[verify] val_loss: tensor(2.0742e-05)
[verify] torch.max: tensor([12.5651]) tensor([0])
[verify] output: tensor([0])
[verify] pred_output: tensor([[ 11.7342, -20.9372,   1.6654]])
[verify] val_loss: tensor(4.2438e-05)
[verify] torch.max: tensor([11.7342]) tensor([0])
[verify] output: tensor([2])
[verify] pred_output: tensor([[-5.6528, -2.0703,  2.3366]])
[verify] val_loss: tensor(0.0125)
[verify] torch.max: tensor([2.3366]) tensor([2])
[verify] output: tensor([2])
[verify] pred_output: tensor([[-6.4438, -1.0253,  1.8898]])
[verify] val_loss: tensor(0.0530)
[verify] torch.max: tensor([1.8898]) tensor([2])
[verify] output: tensor([1])
[verify] pred_output: tensor([[-13.8890,   6.5201,  -1.2541]])
[verify] val_loss: tensor(0.0004)
[verify] torch.max:

[verify] output: tensor([0])
[verify] pred_output: tensor([[ 12.0286, -21.6406,   1.7346]])
[verify] val_loss: tensor(3.3855e-05)
[verify] torch.max: tensor([12.0286]) tensor([0])
[verify] output: tensor([2])
[verify] pred_output: tensor([[-7.0050, -0.2059,  1.4645]])
[verify] val_loss: tensor(0.1726)
[verify] torch.max: tensor([1.4645]) tensor([2])
[verify] output: tensor([2])
[verify] pred_output: tensor([[-8.2644,  2.1752,  0.3881]])
[verify] val_loss: tensor(1.9420)
[verify] torch.max: tensor([2.1752]) tensor([1])
[verify] output: tensor([2])
[verify] pred_output: tensor([[-5.5841, -2.8235,  2.7147]])
[verify] val_loss: tensor(0.0042)
[verify] torch.max: tensor([2.7147]) tensor([2])
[verify] output: tensor([1])
[verify] pred_output: tensor([[-11.4133,   5.2864,  -0.9613]])
[verify] val_loss: tensor(0.0019)
[verify] torch.max: tensor([5.2864]) tensor([1])
[verify] output: tensor([2])
[verify] pred_output: tensor([[-6.9775, -1.2565,  2.1437]])
[verify] val_loss: tensor(0.0329)
[verif